# Sliding Window Decoding

Run both circuit-level and phenomenological sliding-window decoders.


In [1]:
import numpy as np
import stim

from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


### Decoder parameters
- `W`: sliding window width (number of rounds per window)
- `F`: overlap between consecutive windows
- `max_iter`: max BP iterations per window
- `osd_order`: OSD search depth for the BP-OSD decoder


In [2]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 10
num_trials = 100
W, F = 5, 3
max_iter, osd_order = 10, 1


In [3]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'pairing_rank': 9, 'pairing_is_identity': True, 'ok': True}


In [4]:
# Generate memory experiment circuit
circuit = stim.Circuit(
    get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis="Z")
)
check_overlapping_CX(circuit);
# print(circuit)


No overlapping CX gates found.


In [5]:
# Simulate the circuit
detection_events, observable_flips = get_stim_mem_result(
    circuit, num_trials, seed=seed
)
print(detection_events.shape, observable_flips.shape)


(100, 1296) (100, 9)


### Outputs/metrics 
- `pL`: logical error rate per experiment (any logical flip)
- `LFR`: logical failure rate per round, `1 - (1 - pL)^(1/num_rounds)`


In [6]:
# Sliding window circuit-level decoding (BP-OSD)
logical_pred = sliding_window_bposd_circuit_mem(
    detection_events,
    circuit,
    code.hz,
    code.lz,
    W,
    F,
    max_iter=max_iter,
    osd_order=osd_order,
    tqdm_on=True,
)

pL = np.mean((observable_flips - logical_pred).any(axis=1))
lfr = 1 - (1 - pL) ** (1 / num_rounds)
print({"p": p, "pL": float(pL), "lfr": float(lfr)})


100%|██████████| 100/100 [00:12<00:00,  8.06it/s]

{'p': 0.001, 'pL': 0.0, 'lfr': 0.0}


In [7]:
# Sliding window phenomenological decoding (BP-OSD)
depth = sum(code.num_colors.values())
eff_error_rate_per_fault = p * (depth + 3)

logical_pred = sliding_window_bposd_phenom_mem(
    detection_events,
    code.hz,
    code.lz,
    W,
    F,
    eff_error_rate_per_fault=eff_error_rate_per_fault,
    max_iter=max_iter,
    osd_order=osd_order,
    tqdm_on=True,
)

pL = np.mean((observable_flips - logical_pred).any(axis=1))
lfr = 1 - (1 - pL) ** (1 / num_rounds)
print({"p": p, "eff_error_rate_per_fault": float(eff_error_rate_per_fault), "pL": float(pL), "lfr": float(lfr)})


100%|██████████| 100/100 [00:01<00:00, 60.22it/s]

{'p': 0.001, 'eff_error_rate_per_fault': 0.011, 'pL': 0.02, 'lfr': 0.002018231358430378}
